In [1]:
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import Literal
from data_setup import model, retriever

Loading existing Chroma store


In [2]:
# yes or no로 청크 거르는 클래스
class GradeDocuments(BaseModel):
    """검색된 문서가 질문과 관련이 있는지 'yes' 또는 'no로 평가합니다."""

    binary_score: Literal["yes", "no"] = Field(
        # description 생략
    )

structured_llm_grader = model.with_structured_output(GradeDocuments)

In [3]:
# grade 프롬프트 작성
grader_prompt = PromptTemplate.from_template("""
당신은 검색된 문서가 사용자 질문과 관련이 있는지 평가하는 평가자입니다. \n
문서에 사용자 질문과 관련된 키워드 또는 의미가 포함되어 있으면, 해당 문서를 관련성이 있다고 평가하십시오. \n
엄격한 테스트가 필요하지 않습니다. 목표는 잘못된 검색 결과를 걸러내는 것입니다. \n
문서가 질문과 관련이 있는지 여부를 나타내기 위해 'yes' 또는 'no'로 이진 점수를 부여하십시오      

Retrieved document: \n {document} \n\n
User question: {question}                                                                                                                                                                                                                              
""")

retrieval_grader = grader_prompt | structured_llm_grader

In [4]:
question = "서울시 자율주행 관련 계획"
documents = retriever.invoke(question)

for doc in documents:
    print(doc)

page_content='54제2장 미래상과 목표
5. 기술발전에 선제적 대응, ‘미래교통 인프라 구축’1) 배경미래교통수단 도입에 따른 도시 활동 변화 및 공간구조 재편 대비Ÿ빠르게 진화하는 미래 신 교통수단은 도시공간에 새로운 변화를 일으킬 것이며, 도시 내 이동패턴의 변화를 가져올 것으로 예측된다. 이러한 점을 고려하여 도시 내 다양한 통행행태가 공존하는 포용적인 교통체계 구축을 준비해야 한다.미래교통수단 정착을 위한 도시 인프라 확충지원 필요Ÿ기존에 없던 새로운 교통은 통상 인프라 구축에 많은 시간이 소요되기 때문에 교통수단이 도입된 이후에 준비하기보다는 개발단계에서부터 이를 지원하기 위한 교통인프라 구축 구상이 요구되며, 새로운 도시 인프라 확충에 대한 도시계획적 지원이 필요하다.2) 추진전략자율주행 자동차 본격 운영 체계 구축Ÿ자율주행은 현재 제일 가시화되고 있는 미래 교통기술로 시장의 주도권을 확보하기 위해 각국의 노력이 집중되고 있다. 단계적으로 2030년까지 간선도로급 이상 도로에서 운영할 수 있는 도로 인프라 환경을 조성하고, 2040년까지 서울 전역 자율주행 운행 환경 구축을 목표로 한다.-2040년까지 서울 전역 자율주행차량의 운행 환경을 구축하고 수송 분담률 10%를 달성서울형 도심 항공교통 기반 마련 및 터미널 확충Ÿ도심에서 김포공항까지 시범노선을 운영하는 등 상용화 노선을 확보하고, 장기적으로 한강, 지천 등 주요 수변 공간 중심으로 광역노선을 확보한다. 또한 UAM 인프라 확보를 위한 인센티브 및 도시계획적인 지원 방안도 함께 검토한다.-UAM 시범노선을 운영하고, 한강 등 주요 수변 공간 중심에 단계적 광역노선을 확장서울 전역에 모빌리티 허브 구축Ÿ도심 및 광역거점에 서울 내부와 외부의 연계를 위한 미래 교통과 기존 교통을 연결하는 시설을 도입한다. 또한 교통뿐 아니라 공공서비스, 물류, 상업 등 다양한 기능들이 복합적으로 제공되는 지역 거점 모빌리티 허브를 서울 전역에 도입한다.' metadata={'pdfversion': '1

In [5]:
# 관련 청크만 리스트에 추가
filtered_docs = []

for i, doc in enumerate(documents):
    print(f"Document {i+1}:")
    is_relevant = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    print(is_relevant)
    print(doc.page_content[:100])

    if is_relevant.binary_score == "yes":
        filtered_docs.append(doc)

print(f"Filtered documents: {len(filtered_docs)}")

Document 1:
binary_score='yes'
54제2장 미래상과 목표
5. 기술발전에 선제적 대응, ‘미래교통 인프라 구축’1) 배경미래교통수단 도입에 따른 도시 활동 변화 및 공간구조 재편 대비Ÿ빠르게 진화하는 미래 신 교
Document 2:
binary_score='yes'
36제2장 미래상과 목표
6) 미래교통수단의 등장과 첨단 인프라 요구 증대자율주행, UAM 등 미래 교통에 공간적 대응방안 마련 필요Ÿ자율주행차량, 전동킥보드 등 교통 기술의 발전
Document 3:
binary_score='no'
제1절 도시공간구조105기능이 공존하는 미래 도심으로 조성-국제 디지털 금융중심지인 여의도·영등포는 한강을 중심으로 한 글로벌 혁신코어 조성-국제 업무 중심지인 강남은 영동대로와 
Document 4:
binary_score='yes'
광화문·시청역을, GTX-B 노선은 종로3가역 또는 동대문역 신설 제안을 검토Ÿ자전거를 비롯하여 개인용 이동수단(PM), 수상교통, 도심항공교통(UAM) 등 신 이동수단으로 도심 
Document 5:
binary_score='no'
제3절 권역별 구상147보행 중심 교통체계 전환을 위한 친환경 교통 인프라 구축 및 제도 개선 Ÿ주차상한제 강화, 공해 차량 통행 제한 확대, 탄력적인 주차요금제 적용 등으로 도심
Filtered documents: 3


In [6]:
# RAG 답변 생성을 위한 프롬프트 생성
rag_generate_system = """
너는 사용자의 질문에 대해 주어진 context에 기반하여 답변하는 도시 계획 전문가이다.
주어진 context는 vectorstore에서 검색된 결과이다.
주어진 context를 기반으로 사용자의 question에 대해 답변하라.

=============================
question: {question}
context: {context}
"""

rag_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=rag_generate_system,
)

rag_chain = rag_prompt | model

In [7]:
question = "서울시 자율주행 관련 계획"

rag_chain.invoke({"question": question, "context": filtered_docs})

AIMessage(content='서울시는 자율주행 자동차를 포함한 미래 교통수단에 대한 포괄적인 계획을 수립하고 있습니다. 이러한 계획의 주요 목표는 2030년까지 간선 도로 이상에서 자율주행 자동차의 운영을 위한 기본 인프라를 구축하고, 2040년까지 서울 전역에서 자율주행차량이 운행될 수 있는 환경을 조성하는 것입니다. 이를 통해 자율주행차의 수송 분담률을 10%에 달하게 할 계획입니다.\n\n또한, 서울시는 도심 항공교통(UAM)의 기반 마련 및 터미널 확충도 진행할 예정입니다. 시범노선을 운영하고, 한강 및 주요 수변 공간에 광역 노선을 단계적으로 확장할 계획입니다. 이와 함께, 도심 및 광역거점에 모빌리티 허브를 구축하여 다양한 교통수단과 공공서비스, 물류, 상업 기능들이 통합된 장소를 제공할 계획이기도 합니다.\n\n이러한 계획은 자율주행 기술의 발전 및 기타 신교통수단의 필요에 따른 도시 공간 구조의 변화에 적극 대응하고 있으며, 향후 서울의 교통 체계를 더 안전하고 효율적으로 발전시키는데 기여할 것으로 기대됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 276, 'prompt_tokens': 1791, 'total_tokens': 2067, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BrEbUbvgogK4pWAGhsE7xiabnB